In [106]:
# import required libries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
# define required path
curated_data = '../../data/curated/'
raw_data = '../../data/raw/'

In [107]:
liveable = pd.read_csv(curated_data + 'Liveability_rank.csv', index_col = 0)
affordable = pd.read_csv(curated_data + 'Affordability_rank.csv', index_col = 0)

In [108]:
# Code from python tutorial https://pythonguides.com/download-zip-file-from-url-using-python/
# importing necessary modules
import requests, zipfile
from io import BytesIO
print('Downloading started')

#Defining the zip file URL
url = "https://data.gov.au/data/dataset/af33dd8c-0534-4e18-9245-fc64440f742e/resource/4494abe0-64ea-4fa6-931a-d1a389a14e57/download/vic_localities.zip"

# get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)
print('Downloading Completed')
# extracting the zip file contents
zipfile= zipfile.ZipFile(BytesIO(req.content))
zipfile.extractall(raw_data + 'suburbs_shape')

In [109]:
# code from tut02
# sf stands for shape file
sf = gpd.read_file(raw_data + "suburbs_shape/vic_localities.shp")

In [110]:
# Convert the geometry shaape to to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [111]:
# (y, x) since we want (lat, long)
sf['centroid'] = sf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))

In [112]:
# find top 5 affordable suburbs
affordable_suburbs = list(affordable['Suburb'].head(5))

In [113]:
# find top 5 liveable suburbs
liveable_suburbs = list(liveable['Suburb'].head(5))

In [114]:
# prepare for merge
sf['LOC_NAME'] = sf['LOC_NAME'].str.upper()

In [115]:
# select the top 5 affordable suburbs
top5_aff = sf.query('LOC_NAME in @affordable_suburbs')

In [116]:
# select the top 5 liveable suburbs
top5_live = sf.query('LOC_NAME in @liveable_suburbs')

In [117]:
map_aff = folium.Map(location=[-37.02, 146.76], tiles="Stamen Terrain", zoom_start=10)

In [118]:
for zone_name, coord in top5_aff[['LOC_NAME', 'centroid']].values:
    map_aff.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
map_aff.save('../../plots/affordable_Map.html')

In [119]:
map_live = folium.Map(location=[-37.02, 146.76], tiles="Stamen Terrain", zoom_start=10)

In [120]:
for zone_name, coord in top5_live[['LOC_NAME', 'centroid']].values:
    map_live.add_child(
        folium.Marker(location=coord, popup=zone_name)
    )
map_live.save('../../plots/liveable_Map.html')